# House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [166]:
# Use this cell to write your code for Task 1
import pandas as pd

houses = pd.read_csv("house_sales.csv", parse_dates= ["sale_date"])

houses.isna().sum()    # 31 NaNs in months_listed

city_cond = houses["city"].isin(['Silvertown', 'Riverford', 'Teasdale', 'Poppleton'])

missing_city = len(houses[~city_cond])

print(missing_city)

73


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [167]:
# Use this cell to write your code for Task 2
import pandas as pd
import numpy as np

#houses = pd.read_csv("house_sales.csv", parse_dates= ["sale_date"])
houses = pd.read_csv("house_sales.csv")
clean_data = houses.copy()

# Change column names
new_names = {
    'house_id': 'house_id' ,
    'city': 'city',
    'sale_price': 'sale_price',
    'sale_date': 'sale_date' ,
    'months_listed': 'months_listed',
    'bedrooms': 'bedrooms',
    'house_type': 'house_type' ,
    'area': 'area'
}

# Rename columns using the dictionary
clean_data.rename(columns=new_names, inplace=True)

# Fill missing data
city_cond = clean_data["city"].isin(['Silvertown', 'Riverford', 'Teasdale', 'Poppleton'])

# fill city '--' missing values with "Unkown"
clean_data.loc[~city_cond, "city"] = None
clean_data["city"].fillna("Unknown", inplace= True)

# sale_price: drop NaNs entries
clean_data.dropna(subset=['sale_price'], inplace= True)

# sale_date: fill with "2023-01-01"
#clean_data["sale_date"] = clean_data["sale_date"].dt.date
clean_data["sale_date"].fillna("2023-01-01")

# months_mean: fill with mean rouded to 1 decimal, and round all rows to 1 deciaml
months_mean = np.round(clean_data["months_listed"].mean(), 1)
clean_data["months_listed"].fillna(months_mean, inplace= True)
clean_data["months_listed"] = np.round(clean_data["months_listed"], 1)

# bedrooms: replace missing values with the mean number of bedrooms, rounded to the nearest integer
bedrooms_mean = np.round(clean_data["bedrooms"].mean())
clean_data["bedrooms"].fillna(bedrooms_mean, inplace= True)


# house_type: 
    # 1.first collapse similar categories into the correct ones
mapping = {
    'Det.': 'Detached',
    'Semi': 'Semi-detached',
    'Terr.': 'Terraced'
}
clean_data['house_type'] = clean_data['house_type'].replace(mapping)
#clean_data["house_type"].unique()

    # 2. replace missing values with the most common house type.
most_common = clean_data["house_type"].value_counts(sort=True).idxmax()
clean_data["house_type"].fillna(most_common, inplace=True)

# area: Replace missing values with the mean, to one decimal place. Round all rows to one decimal place
    # 1. clean string data and convert to numeric
clean_data["area"]= clean_data["area"].str.strip("sq.m.")
clean_data["area"]= clean_data["area"].astype(float)

    # 2. fill NaNs with mean
area_mean = np.round(clean_data["area"].mean(), 1)
clean_data["area"].fillna(area_mean, inplace=True)

# Change data types

    # Set house_type category order
category_order = ['Terraced', 'Semi-detached', 'Detached']

    # Convert 'house_type' column to ordinal categorical data
clean_data['house_type'] = pd.Categorical(clean_data['house_type'], 
                                          categories=category_order, 
                                          ordered=True)

types = {
    'house_id': 'int64' ,
    'city': 'category',
    'sale_price': 'int64',
    'months_listed': 'float64',
    'bedrooms': 'int64',
    'area': 'float64'
}
clean_data = clean_data.astype(types)

#print(clean_data.info())

#for column in list(clean_data.columns):
#    print("\nUnique Values for {}:".format(column))
#    print(clean_data[column].unique())

# Display resulting dataframe
print(clean_data)

      house_id        city  sale_price  ... bedrooms     house_type   area
0      1217792  Silvertown       55943  ...        2  Semi-detached  107.8
1      1900913  Silvertown      384677  ...        5       Detached  498.8
2      1174927   Riverford      281707  ...        6       Detached  542.5
3      1773666  Silvertown      373251  ...        6       Detached  528.4
4      1258487  Silvertown      328885  ...        5       Detached  477.1
...        ...         ...         ...  ...      ...            ...    ...
1495   1123892   Riverford      198661  ...        5       Detached  432.2
1496   1327295   Poppleton      358304  ...        6       Detached  599.8
1497   1058161   Riverford      176612  ...        4       Detached  359.1
1498   1822037    Teasdale      197827  ...        3       Detached  253.7
1499   1679272   Poppleton      162099  ...        3       Detached  268.7

[1500 rows x 8 columns]


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [168]:
# Use this cell to write your code for Task 3

houses = pd.read_csv("house_sales.csv")

price_by_rooms = houses.groupby("bedrooms")["sale_price"].agg(['mean', 'var']).reset_index()

column_names = {'mean': 'avg_price', 'var': 'var_price'}
price_by_rooms = np.round(price_by_rooms.rename(columns = column_names), 1)

price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [169]:
# Use this cell to write your code for Task 4

train = pd.read_csv('train.csv')
test = pd.read_csv('validation.csv')

#train_clean = data_cleaner(train)
#test_clean = data_cleaner(test)
train_clean = train
test_clean = test
#print(train_clean.equals(train))
#print(test_clean.equals(test))

        # All data seem to be already clean

#for column in list(test.columns):
#    print("\nUnique Values for {}:".format(column))
#    print(test[column].unique())
    
#test.isna().sum()

def transform_data(data):
    new_data= data.copy()
    
    # Drop iD column
    new_data = new_data.drop("house_id", axis=1)
    
    # Get dummy variables for city and house_type
    new_data = pd.get_dummies(new_data, columns = ["city", "house_type"])
    
    # Extract useful information from sale_date column
    new_data["sale_date"] = pd.to_datetime(new_data['sale_date'])
    new_data["year"] = new_data["sale_date"].dt.year
    new_data["month"] = new_data["sale_date"].dt.month
    new_data["day"] = new_data["sale_date"].dt.day
    
    new_data = new_data.drop("sale_date", axis=1)
            
    return new_data
        
train_clean = transform_data(train_clean)
test_clean = transform_data(test_clean)

# Split train_clean into X_train and y_train
X_train = train_clean.drop("sale_price", axis=1).values
y_train = train_clean["sale_price"].values

X_test = test_clean.values

# Create and fit model
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

base_result = pd.DataFrame({'house_id': test["house_id"], 'price': y_pred})

# Display the resulting dataframe
print(base_result)


     house_id          price
0     1331375  121450.319429
1     1630115  304384.498235
2     1645745  384629.788665
3     1336775  124804.725941
4     1888274  271216.164572
..        ...            ...
295   1986255  349810.906432
296   1896276  369221.549578
297   1758223  257517.739299
298   1752010  169452.990144
299   1651404  389009.461825

[300 rows x 2 columns]


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [170]:
# Use this cell to write your code for Task 5

train = pd.read_csv('train.csv')
test = pd.read_csv('validation.csv')

#train_clean = data_cleaner(train)
#test_clean = data_cleaner(test)
train_clean = train
test_clean = test
#print(train_clean.equals(train))
#print(test_clean.equals(test))

        # All data seem to be already clean

def transform_data(data):
    new_data= data.copy()
    
    # Drop iD column
    new_data = new_data.drop("house_id", axis=1)
    
    # Get dummy variables for city and house_type
    new_data = pd.get_dummies(new_data, columns = ["city", "house_type"])
    
    # Extract useful information from sale_date column
    new_data["sale_date"] = pd.to_datetime(new_data['sale_date'])
    new_data["year"] = new_data["sale_date"].dt.year
    new_data["month"] = new_data["sale_date"].dt.month
    new_data["day"] = new_data["sale_date"].dt.day
    
    new_data = new_data.drop("sale_date", axis=1)
            
    return new_data
        
train_clean = transform_data(train_clean)
test_clean = transform_data(test_clean)

# Split train_clean into X_train and y_train
X_train = train_clean.drop("sale_price", axis=1).values
y_train = train_clean["sale_price"].values

X_test = test_clean.values

# Create and fit model
from sklearn.ensemble import RandomForestRegressor

model_2 = RandomForestRegressor()
model_2.fit(X_train, y_train)

y_pred_2 = model_2.predict(X_test)

compare_result = pd.DataFrame({'house_id': test["house_id"], 'price': y_pred_2})

# Display the resulting dataframe
print(compare_result)

     house_id      price
0     1331375   81382.84
1     1630115  306317.98
2     1645745  400012.88
3     1336775  106068.69
4     1888274  267081.75
..        ...        ...
295   1986255  358878.35
296   1896276  385318.45
297   1758223  262584.98
298   1752010  175009.76
299   1651404  415645.21

[300 rows x 2 columns]
